# Joint Intent detection and slot filling
Dieses Jupyter notebook wurde als semesterabschließende Arbeit für das Modul Natural Language Processing an der [Fachhochschule Südwestfalen](https://www.fh-swf.de/en/international_3/index.php) erstellt.

## Einleitung
Das Joint intent detection and slot filling (IDSF) ist eine Aufgabe aus dem Teilbereich des Natural Language Understandings (NLU) des Natural Language Processings (NLP), die uns in heutzutage fast täglich Alltag begegnet. Sei es um einen Timer auf dem Handy zu starten, bestimmte Musik abzuspielen oder das Licht einzuschalten. Der Ablauf ist dabei häufig der selbe: "Siri stelle einen Timer für 4 Minuten", "Alexa spiele meine Schlager Playlist" oder "Google erstelle einen Arzttermin für heute 16:00 Uhr". Meist beginnen die Kommandos mit dem Namen des Sprachassistenten, um diesen zu aktivieren, gefolgt vom Kommando für die gewünschte Aktion. Das IDSF beschäftigt sich dabei mit der Aufgabe, die gewünschte Aktion (Intent), also stelle einen Timer, Spiele Musik, erstelle einen Termin im Kalender und die dazugehörigen notwendigen Parameter, wie z.B. vier Minuten, Schlager Playlist oder Arzt heute 16:00 Uhr (Slots) zu erkennen.
Da der Gebrauch dieser Sprachassistenten in Zukunft wahrscheinlich noch stärker zu nehmen wird, wollen wir uns deren funktionsweise in diesem Notebook näher anschauen. Dafür wird zuerst die Entwicklungshistorie vom IDSF betrachtet und anschließend wird ein eigenes Modell für die Erkennung erstellt und anhand eines selbst vorbereiteten Korpus trainiert.

## Joint Intent Detection and Slot filling

--------------

## Datenbeschaffung

Als Datensatz für das nachfolgende Beispiel verwenden wir den Snips-Datensatz. Dieser Datensatz wurde vom, mittlerweil zu Sonos gehörenden [1], [Snips Team](https://snips.ai/) zusammengestellt, um ihr eigenes Modell mit anderen Wettbewerbern wie zum Beispiel Amazons Alexa zu verglichen. Die Ergebnisse und die Datensätze der drei Vergleiche wurden in einem [GitHub Repository](https://github.com/sonos/nlu-benchmark/tree/master) veröffentlicht und in dem Paper "Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces" [2] erläutert. Das Repository enthält die Daten für drei Evaluationen aus den Jahren 2016 bis 2018. Wir werden in diesem Notebook die Daten der 2017 durchgeführten Evaluation verwenden, da diese Sätze für sieben unterschiedliche und allgemeine Aufgaben enthält.

Die Daten sind im dem Repository in einzelnen JSON-Dateien enthalten. Dabei gibt es pro Aufgabe zwei Dateien, eine für das Training und eine für die Validierung. Der Einfachheit halber wurden die Dateien in dem data Verzeichnis, dass diesem Notebook beiliegt, abgelegt.

Nachfolgend ist ein Auszug aus der `train_AddToPlaylist_full.json`-Datei.

In [15]:
!head -n 48 data/train_AddToPlaylist_full.json # Zeige die ersten 23 Zeilen der angegebenen Datei an.

{
  "AddToPlaylist": [
    {
      "data": [
        {
          "text": "Add another "
        },
        {
          "text": "song",
          "entity": "music_item"
        },
        {
          "text": " to the "
        },
        {
          "text": "Cita Romántica",
          "entity": "playlist"
        },
        {
          "text": " playlist. "
        }
      ]
    },
    {
      "data": [
        {
          "text": "add "
        },
        {
          "text": "clem burke",
          "entity": "artist"
        },
        {
          "text": " in "
        },
        {
          "text": "my",
          "entity": "playlist_owner"
        },
        {
          "text": " playlist "
        },
        {
          "text": "Pre-Party R&B Jams",
          "entity": "playlist"
        }
      ]
    },


Die Datei besteht an oberster Stelle aus dem Namen der gewünschten Aktion gefolgt von einer Liste an Objekten mit einem `Data` Attribut. Dieses enthält wiederum eine Liste von Objekten mit `Text` Attributen die den Satz in Teilen enthält. Dabei wird der Satz durch den Text eines definierten `Entities` geteilt. So enthält das erste Beispiel den Text bis zum ersten `entity` das als `music_item` klassifiziert wurde und wieder den gesamten Text bis zum nächsten entity, dem Namen einer Playlist.

Als nächstes werden die Daten in ein verwendbares Format transformiert. Ein in der Natural language processing gängiges Format ist das IOB Format. IOB steht für Inside-Outside-Beginning. Dieses Format ermöglicht die Kennzeichnung der einzelnen Entitäten in einem Satz. Es wird unteranderem von den weit verbreiteten Python Bibliotheken `NLTK` und `spaCy` unterstützt [3, 4]. Das Format wurde 1995 von Lance A. Ramshaw und Mitchell P. Marcus erfunden.

Dieses Beispiel zeigt das Format einer Zeile, welches nachfolgend aus den JSON-Dateien erzeugt wird. Die IOB-Kodierung ist hinter der Intent-Kategorie `AddToPlaylist` zu sehen.

    BOS add clem burke in my playlist Pre-Party R&B Jams EOS AddToPlaylist o o b-mucic_item i-music_item o i-playlist_owner o b-playlist i-playlist i-playlist
    
Am Beginn der Zeile steht der vollständige Satz abgetrennt durch ein BOS (begin of sentence) am Anfang des Satzes und ein EOS (end of sentence) am Ende des Satzes. Dahinter wird die Aktion (Intent) definiert. Nun folgt das eigentliche IOB-Format. Dabei wird für jeden Token entweder der Buchstabe 'o', dieser steht für keine Bedeutung, der Buchstabe 'b', für den Beginn einer Entität die aus mehreren Token besteht, oder 'i', als Entität. Das 'i' steht dabei entweder nach einem 'b' wodurch eine Entität gekennzeichent wird, die aus mehreren Token besteht oder alleine für eine Entität die aus nur einem Token besteht.  Das 'b' und 'i' werden dabei jeweils gefolgt vom einem trennenden Bindestrich und der Entitätskategorie verwender. So ist der Name 'Clem Burke' unterteilt in ein `b-music_item` für Clem und `i-Music_item` für Burke. Dadurch wird definiert, dass die beiden Teile zusammen gehören.

Das IOB2 Format ist eine Erweiterung des originalen IOB Formats. Es definiert das auch eine Entität die nur aus einem Token besteht mit einem 'b' kodiert wird und nicht wie im IOB Format mit einem 'i'. Dadurch ergibt sich das folgende Format:

    BOS add clem burke in my playlist Pre-Party R&B Jams EOS AddToPlaylist o o b-mucic_item i-music_item o b-playlist_owner o b-playlist i-playlist i-playlist
    

Mit dem folgenden Python Code wird der Inhalt der im data-Verzeichnis liegenden Dateien in das vorgestellte Format transformiert.


In [16]:
import os
import json
import re
from shutil import rmtree

In [17]:
def clean_formatted(formatted_directory_path):
    if not os.path.exists(formatted_directory_path) or not os.path.isdir(formatted_directory_path):
        print(f'Pfad {formatted_directory_path} ist kein Verzeichnis oder existiert nicht')
        return

    rmtree(formatted_directory_path)
    print('Alte Corpus-Dateien gelöscht')

def convert_file(json_file_path):
    print(f'Try converting file at path {json_file_path}')
    if not os.path.isfile(json_file_path):
        print(f'File {file_path} does not exists', json_file_path)
        return 
    formatted_lines = []
    intent_category = None
    with open(json_file_path, 'r', encoding='latin-1') as json_file:
        json_content = json.load(json_file)
        intent_category = next(iter(json_content))
        for sentence_block in json_content[intent_category]:
            sentence = ""
            slots = []
            for sentence_data_block in sentence_block['data']:
                sentence_part = sentence_data_block['text']
                sentence_part = re.sub('\n', '', sentence_part)
                if sentence_part != '':
                    sentence += sentence_part
                sentence_part_len = len(sentence_part.split())
                if 'entity' in sentence_data_block:
                    entity_type = sentence_data_block['entity']
                    if sentence_part_len > 1:
                        firstSlot = True
                        for i in range(sentence_part_len):
                            if firstSlot:
                                slots.append('b-' + entity_type)
                                firstSlot = False
                            else:
                                slots.append('i-' + entity_type)
                    else:
                        slots.append('b-' + entity_type)
                else:
                    for i in range(sentence_part_len):
                        slots.append('o')
            formatted_lines.append(construct_row(sentence, intent_category, slots))
    print(f'Finished converting file at path {json_file_path}. Writing to file...')
    write_to_file(intent_category, formatted_lines)
    

def construct_row(sentence, intent, slots):
    row = 'BOS '
    row += sentence
    row += ' EOS '
    row += intent
    row += ' '
    row += ' '.join(slots)
    row += '\n'
    return row


def write_to_file(intent, lines):
    if intent is None or intent == '':
        print('No intent')
        return
        
    base_output_directory = 'data/formatted/'
    output_file_path = base_output_directory + intent + '.txt'

    if not os.path.exists(base_output_directory): 
        os.makedirs(base_output_directory)
    
    with open(output_file_path, 'a') as output_file:
        output_file.writelines(lines)


def iterate_over_json_files_in_directory(directory_path):
    if not os.path.exists(directory_path) or directory_path is not os.path.isdir(directory_path):
        print(f"Das Pfad {directory_path} existiert nicht oder ist kein Verzeichnis.")
        
    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):  # Nur JSON-Dateien berücksichtigen
            file_path = os.path.join(directory_path, filename)
            convert_file(file_path)
    print('Finished converting all files!')
    
clean_formatted('data/formatted')
iterate_over_json_files_in_directory('data')

Pfad data/formatted ist kein Verzeichnis oder existiert nicht
Das Pfad data existiert nicht oder ist kein Verzeichnis.
Try converting file at path data/validate_PlayMusic.json
Finished converting file at path data/validate_PlayMusic.json. Writing to file...
Try converting file at path data/train_SearchCreativeWork_full.json
Finished converting file at path data/train_SearchCreativeWork_full.json. Writing to file...
Try converting file at path data/train_AddToPlaylist_full.json
Finished converting file at path data/train_AddToPlaylist_full.json. Writing to file...
Try converting file at path data/train_RateBook_full.json
Finished converting file at path data/train_RateBook_full.json. Writing to file...
Try converting file at path data/train_SearchScreeningEvent_full.json
Finished converting file at path data/train_SearchScreeningEvent_full.json. Writing to file...
Try converting file at path data/validate_GetWeather.json
Finished converting file at path data/validate_GetWeather.json. Wr

Als nächstes wird geprüft, ob auch alle Einträge in der Textdatei enthalten sind. Dafür werden die Einträge in der train- und validate.json mit der Anzahl der Zeilen in der Textdatei verglichen.

In [18]:
!wc -l data/formatted/RateBook.txt

2056 data/formatted/RateBook.txt


In [19]:
!jq '.RateBook | length' data/train_RateBook_full.json

1956


In [20]:
!jq '.RateBook | length' data/validate_RateBook.json

100


Man sieht, dass die Anzahl der `data`-Blöcke aus den JSON-Dateien der Anzahl der Zeilen in der erzeugten Textdatei entspricht. Die Konvertierung war also erfolgreich.

In [21]:
!head -n 10 data/formatted/RateBook.txt

BOS rate The Lotus and the Storm zero of 6 EOS RateBook o b-object_name i-object_name i-object_name i-object_name i-object_name b-rating_value o b-best_rating
BOS Rate The Fall-Down Artist 5 stars. EOS RateBook o b-object_name i-object_name i-object_name b-rating_value b-rating_unit o
BOS Rate the current novel one points EOS RateBook o o b-object_select b-object_type b-rating_value b-rating_unit
BOS rate The Ape-Man Within 4 EOS RateBook o b-object_name i-object_name i-object_name b-rating_value
BOS I give The Penalty three stars EOS RateBook o o b-object_name i-object_name b-rating_value b-rating_unit
BOS rate this novel a 4 EOS RateBook o b-object_select b-object_type o b-rating_value
BOS give 5 out of 6 points to Absolutely, Positively Not series EOS RateBook o b-rating_value o o b-best_rating b-rating_unit o b-object_name i-object_name i-object_name b-object_part_of_series_type
BOS I give Emile, or On Education five points. EOS RateBook o o b-object_name i-object_name i-object_nam

Quellen

* 1: https://investors.sonos.com/news-and-events/investor-news/latest-news/2019/Sonos-Announces-Acquisition-of-Snips/default.aspx, [Online, 07.03.2025]
* 2: Coucke A. et al., "Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces." 2018, [Online: https://arxiv.org/abs/1805.10190, 07.03.2025]
* 3: NLTK Team, tree2conlltags, [Online: https://www.nltk.org/_modules/nltk/chunk/util.html#tree2conlltags, 13.03.2025]
* 4: Explosion, spaCy convert, [Online: https://spacy.io/api/cli#converters, 13.03.2025]
* 5: Ramshaw und Marcus, "Text Chunking using Transformation-Based Learning" 1995, [Online: https://arxiv.org/abs/cmp-lg/9505040, 14.03.2025]

## Erstellen eines Corpus